# Training CNN-3, CNN-6 and CNN-8 on Private Data without B/W-tests

In [1]:
import numpy as np
from numpy import array
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

import math

import matplotlib.pylab as plt
from matplotlib.pyplot import cm
import seaborn as sns
import itertools
import collections
from collections import OrderedDict, Counter

import scipy.stats as stats
from scipy.stats import normaltest, kstest, ttest_ind, ttest_rel, mannwhitneyu, wilcoxon, levene, bartlett
from scipy.stats import chi2_contingency, fisher_exact, mode, pearsonr, f_oneway, kruskal, spearmanr

from rfpimp import *

from datetime import datetime
import re
from seaborn import heatmap
import random
import statsmodels.distributions.empirical_distribution as edf
from scipy.interpolate import interp1d
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.utils import resample
from functools import reduce
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

from sklearn import metrics, preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, log_loss, recall_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, PowerTransformer

from sklearn.utils import check_consistent_length, column_or_1d, assert_all_finite
from sklearn.utils.extmath import stable_cumsum
from sklearn.utils.multiclass import type_of_target
from sklearn.linear_model import LogisticRegression
import re
import gc

import os

#from sklearn.externals import joblib
import joblib

from importlib import reload
import Filter_and_Grid_Search
Filter_and_Grid_Search = reload(Filter_and_Grid_Search)
from Filter_and_Grid_Search import stratified_split
from Filter_and_Grid_Search import attributes_list, attributes_list_new
from Filter_and_Grid_Search import get_s_stat, get_PSI_stat, get_stats_by_month, get_stats, stable_unstable
from Filter_and_Grid_Search import stable_unstable_by_month_divide, union_datas, individual_hists_all 
from Filter_and_Grid_Search import paired_time_hists_by_month, statistics_with_target
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import statistics_with_target, attributes_list, attributes_list_new, make_standard
from Filter_and_Grid_Search import data_preprocessing_train, data_preprocessing_test
from Filter_and_Grid_Search import receive_correlations, find_doubles_corr
from Filter_and_Grid_Search import stratified_split, two_forests, turn_variables_with_values
from Filter_and_Grid_Search import find_meta_params, calculate_vif#, find_meta_params_mem
from Filter_and_Grid_Search import plot_meta_2d, data_preprocessing, find_ouliers_iqr
from Filter_and_Grid_Search import train_model_receive_stats, simple_b_score_risk
from Filter_and_Grid_Search import max_prof_corve, by_month_gini, check_attribute_list_cases

from Filter_and_Grid_Search import to_zip, br_correction, br_stat

from statsmodels.stats.proportion import proportions_ztest

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import pathlib
import psutil

C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
PATH_DATA = 'D:/Share/01 -Risk Desc Science/AntiFraud/Data Sets/'
PATH = 'D:/Share/safanasev/Python-notebook/AF_ML_v2_2014/'
print(PATH_DATA)
print(PATH)

D:/Share/01 -Risk Desc Science/AntiFraud/Data Sets/
D:/Share/safanasev/Python-notebook/AF_ML_v2_2014/


## Tensor

In [8]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 29

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.compat.v1.set_random_seed(seed_value)


# 5. Configure a new global `tensorflow` session
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [9]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Activation, Dense, Flatten, Dropout)
from tensorflow.keras.layers import (Conv1D, MaxPooling1D, AveragePooling1D)
from tensorflow.keras.layers import add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from keras.utils import np_utils

In [10]:
from tensorflow.keras.layers import ReLU, Concatenate, GlobalAveragePooling1D

In [11]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6806930242532557001
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 5163500694205374332
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2692889278196551721
physical_device_desc: "device: XLA_GPU device"
]


In [12]:
#если alpha не задано, то не делаем br_correction

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,  dataset, x_col, y_col,  batch_size=256, num_classes=2, alpha = 0.1, random_state = 42, 
                 shuffle=True, class_w = None):
        self.batch_size = batch_size
        self.dataset = dataset
        self.x_col = x_col
        self.y_col = y_col
        self.indices = self.dataset.index.tolist()
        self.num_classes = num_classes
        self.shuffle = shuffle
        self.alpha = alpha
        self.random_state = random_state
        self.class_w = class_w
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.indices) / self.batch_size)

    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index + 1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        data_tmp = self.dataset.loc[batch]
        data_tmp_b = data_tmp[data_tmp[self.y_col] == 1]
        data_tmp_b_all = self.dataset[self.dataset[self.y_col] == 1]
        X_tmp = data_tmp.head(0)
        
        
        #print(data_tmp_b.shape[0], data_tmp.shape[0], data_tmp_b_all.shape[0])
        if (self.alpha is None) and (self.class_w is None):
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col] 
            
        elif (self.alpha is None) and self.class_w > 0:
            k = (self.class_w*self.batch_size)/(1-self.class_w)
            k = k - data_tmp_b.shape[0]
            if int(k) < 1:
                k = k+1
            ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
            X_tmp = data_tmp_b_all.iloc[ix2]
            data_tmp = data_tmp.append(X_tmp, ignore_index=True)
            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]
            #print(data_tmp_b.shape[0], data_tmp.shape[0],  data_tmp_b.shape[0]/(data_tmp.shape[0] - X_tmp.shape[0]), np.mean(y)) 
            
            
            
        else:
            if data_tmp_b.shape[0] > 0:
                k=(self.alpha*self.batch_size/data_tmp_b.shape[0] -1 )/(1-self.alpha)
                for i in range(0,int(k)):
                    X_tmp = X_tmp.append(data_tmp_b, ignore_index=True)
            else:
                k = (self.alpha*self.batch_size)/(1-self.alpha)
                ix2 = np.random.RandomState(self.random_state + int(k)).choice(data_tmp_b_all.shape[0], int(k))
                X_tmp = data_tmp_b_all.iloc[ix2]
                data_tmp_b = X_tmp

    #         print('Добавим дробное число строк')    
            k_fraction = k - int(k)
            n_samples = int(round(data_tmp_b.shape[0]*k_fraction))
    #         print(k_fraction, n_samples)

            ix = np.random.RandomState(self.random_state).choice(data_tmp_b.shape[0], n_samples)
            data_add_fraction = data_tmp_b.iloc[ix]
            X_tmp.append(data_add_fraction, ignore_index=True)

            data_tmp = data_tmp.append(X_tmp, ignore_index=True)


            X = data_tmp[self.x_col] #.reshape(-1)
            y = data_tmp[self.y_col]   
        #print(sum(self.dataset.loc[batch][self.y_col])/len(index), np.mean(y))
        
        if self.num_classes > 1:
            y = np_utils.to_categorical( y, self.num_classes)
            
        ####for tf 2.3.0, [np.array(X)], np.array(y)

        #return [np.array(X).reshape(X.shape[0], X.shape[1], 1)], np.array(y)
        return [np.array(X)], np.array(y)

    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)

In [13]:
from tensorflow.keras.callbacks import EarlyStopping, Callback

# объявляем класс метрик

class E_time(Callback):
    def on_train_begin(self, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('train_begin', '| time: ' , tm)
        
    def on_epoch_end(self, epoch, logs={}):
        tm = datetime.strftime(datetime.now(), "%d.%m.%Y %H:%M:%S")
        print ('epoch_end', '| time: ' , tm)
    
        return

_time = E_time()

In [14]:
class Metrics(tf.keras.callbacks.Callback):
    def __init__(self, valid_data):
        super(Metrics, self).__init__()
        self.validation_data = valid_data

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        val_predict = self.model.predict(self.validation_data[0])[:, 1]
        val_targ = self.validation_data[1]
        
        if len(val_targ.shape) == 2: #and val_targ.shape[1] != 1:
            val_targ = val_targ[:,1]

        _val_aps = metrics.average_precision_score(val_targ, val_predict)
        #_val_recall = metrics.recall_score(val_targ, val_predict)
        _val_a = metrics.roc_auc_score(val_targ, val_predict)

        logs['val_aps'] = _val_aps
        logs['val_a'] = _val_a
        print(" — val_aps:  %f — val_a: %f" % (_val_aps, _val_a))
        return

## Loading preprocessed datasets

In [15]:
train_for = pd.read_csv(PATH + 'train_163_prep.csv')
valid_for = pd.read_csv(PATH + 'valid_163_prep.csv')
test_for = pd.read_csv(PATH + 'test_163_prep.csv')

In [16]:
y_train = train_for[target]
y_test = valid_for[target]
y_val = test_for[target]

train_for.drop('Unnamed: 0', axis = 1, inplace = True)
valid_for.drop('Unnamed: 0', axis = 1, inplace = True)
test_for.drop('Unnamed: 0', axis = 1, inplace = True)


col = train_for.columns.to_list()
col.remove(target)


X_1_2 = train_for[col]
X_2_2 = valid_for[col]
X_3_2 = test_for[col]

In [17]:
X_2_2.columns

Index(['COUNT_CONT_30', 'W_TEST_AGE_6_60D', 'W_TEST_AGE_ALL5_7D',
       'B_TEST_PASP_D_3_7D', 'B_TEST_PASP_D_3_14D', 'B_TEST_PASP_D_3_30D',
       'B_TEST_PASP_D_3_60D', 'B_TEST_PASP_D_10_7D', 'B_TEST_PASP_D_10_30D',
       'B_TEST_PASP_D_ALL5_30D',
       ...
       'DIVISION_Южный', 'AVG_FUND_TT_7_bin', 'AVG_FUND_TT_14_bin',
       'STD_SUM_TT_14_bin', 'AVG_FUND_TT_30_bin', 'STD_SUM_TT_30_bin',
       'STD_SUM_TT_60_bin', 'RULE_1_TRE3_SHARE_7_bin',
       'RULE_1_TRE3_SHARE_14_bin', 'COUNT_CONT_FIO7_7_bin'],
      dtype='object', length=163)

In [18]:
x_col = X_2_2.columns
y_col = 'BAD_FLAG'

In [19]:
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )

In [20]:
from keras.utils import np_utils

In [21]:
Y_test_2 = np_utils.to_categorical( y_test, 2) # преобразовываем в 2 класса 

In [22]:
len(x_col)

163

In [63]:
b_test = []
w_test = []
del_all = []

for i in X_2_2.columns:
    if i.find('W_TEST') >= 0:
        w_test.append(i)
        del_all.append(i)
    if i.find('B_TEST') >= 0:
        b_test.append(i)
        del_all.append(i)

In [64]:
col = train_for.columns.to_list()
col.remove(target)

col_2 = [i for i in col if i not in del_all]


X_1_2 = train_for[col_2]
X_2_2 = valid_for[col_2]
X_3_2 = test_for[col_2]

In [65]:
train_for.shape

(1507599, 164)

In [66]:
col_3 = col_2 + [target]

In [67]:
train_for = train_for[col_3]

In [68]:
pd_col = pd.DataFrame()
pd_col['var_wo_btest'] = col_3

In [48]:
pd_col

,var_wo_btest
0,COUNT_CONT_30
1,AVG_SUM_TT_7
2,AVG_FUND_TT_7
3,AVG_SUM_TT_14
4,STD_SUM_TT_14
...,...
120,STD_SUM_TT_60_bin
121,RULE_1_TRE3_SHARE_7_bin
122,RULE_1_TRE3_SHARE_14_bin
123,COUNT_CONT_FIO7_7_bin


In [49]:
pd_col.to_csv('124+target_wo_btest.csv')

In [31]:
def max_pool(min_pool, n_pool, padding_pool,  str_pool, y, str_min = 1):
    #for 3D-tensor
    if y.shape[1] < (min_pool):
        
        return MaxPooling1D(pool_size = n_pool, padding=padding_pool, strides=str_min)(y)
    else:
        return MaxPooling1D(pool_size = n_pool, padding=padding_pool, strides=str_pool)(y)

## CNN

In [32]:
def model_cnn_3(n_features=163, n_pool=2, n_kernel=5, n_filters=5,
                n_strides=1, n_strides_pool=None, classes=2, hidden=64, drop_out=0.25,
                l2_conv=None, reg=1E-5, reg_dense=1E-5, _bias=True, gl_pool_max=False, padding_pool='valid',
                padding_conv = 'same', n_lay=3, min_pool = 15):

    if reg == None:
        l2_batch_gamma = None
        l2_batch_betta = None
    else:
        l2_batch_gamma = l2(reg)
        l2_batch_betta = l2(reg)

    if reg_dense == None:
        kernel_regularizer = None
    else:
        kernel_regularizer = l2(reg)

    # 1 слой

    x = Input(shape=(n_features, 1))
    y = Conv1D(filters=n_filters, kernel_size=n_kernel,
               strides=n_strides, padding=padding_conv, use_bias=_bias)(x)

    y = BatchNormalization(gamma_regularizer=l2_batch_gamma,
                           beta_regularizer=l2_batch_betta)(y)
    y = ReLU()(y)
    y = max_pool(min_pool, n_pool, padding_pool, n_strides_pool, y)

    for i in range(1, n_lay):
        y = Conv1D(filters=(i + 1)* n_filters, kernel_size=n_kernel,
                   strides=n_strides, padding=padding_conv, use_bias=_bias)(y)
        y = BatchNormalization(gamma_regularizer=l2_batch_gamma,
                           beta_regularizer=l2_batch_betta)(y)
        y = ReLU()(y)
        y = max_pool(min_pool, n_pool, padding_pool, n_strides_pool, y)
    if gl_pool_max:
        z = GlobalMaxPooling1D()(y)
        
    else:
        z = GlobalAveragePooling1D()(y)
        
    z = Dense(hidden, activation='relu', kernel_regularizer = kernel_regularizer)(z)
    z = Dropout(drop_out)(z)
    z = Dense(hidden, activation='relu', kernel_regularizer = kernel_regularizer)(z)
    z = Dropout(drop_out)(z)
    predictions = Dense(classes, activation='softmax')(z)

    model = Model(inputs=x, outputs=predictions)


    return model

## CNN-3 test

In [33]:
model_test = model_cnn_3()

In [34]:
model_test.compile(loss='categorical_crossentropy',
                   #optimizer = tf.keras.optimizers.Ftrl(learning_rate=0.003, learning_rate_power=-0.5,  l2_regularization_strength=0.3) ,
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                   #optimizer = tf.keras.optimizers.Adamax(learning_rate=0.006) ,
                   #optimizer=tf.keras.optimizers.Adam(learning_rate=0.002), - не уменьшался loss
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

model_test.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 163, 1)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 163, 5)            30        
_________________________________________________________________
batch_normalization (BatchNo (None, 163, 5)            20        
_________________________________________________________________
re_lu (ReLU)                 (None, 163, 5)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 81, 5)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 81, 10)            260       
_________________________________________________________________
batch_normalization_1 (Batch (None, 81, 10)           

## CNN-6 test

In [46]:
model_test = model_cnn_3(n_lay=6)
model_test.compile(loss='categorical_crossentropy',
                   #optimizer = tf.keras.optimizers.Ftrl(learning_rate=0.003, learning_rate_power=-0.5,  l2_regularization_strength=0.3) ,
                   optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),
                   #optimizer = tf.keras.optimizers.Adamax(learning_rate=0.006) ,
                   #optimizer=tf.keras.optimizers.Adam(learning_rate=0.002), - не уменьшался loss
              metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

model_test.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 163, 1)]          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 163, 5)            30        
_________________________________________________________________
batch_normalization_10 (Batc (None, 163, 5)            20        
_________________________________________________________________
re_lu_10 (ReLU)              (None, 163, 5)            0         
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 81, 5)             0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 81, 10)            260       
_________________________________________________________________
batch_normalization_11 (Batc (None, 81, 10)           

In [59]:
param = itertools.product(gl_p, l2_batch, n_ker, n_fil, d_hidden, drop_out, reg_dense, min_pool, n_lay)

In [56]:
i = 0
for p in param:
    print(i, p)
    i += 1

0 (False, 0.0002, 5, 5, 60, 0.25, 0.0002, 20, 3)
1 (False, 0.0002, 5, 5, 60, 0.25, 0.0002, 20, 6)
2 (False, 0.0002, 5, 5, 100, 0.25, 0.0002, 20, 3)
3 (False, 0.0002, 5, 5, 100, 0.25, 0.0002, 20, 6)
4 (False, 0.0002, 5, 15, 60, 0.25, 0.0002, 20, 3)
5 (False, 0.0002, 5, 15, 60, 0.25, 0.0002, 20, 6)
6 (False, 0.0002, 5, 15, 100, 0.25, 0.0002, 20, 3)
7 (False, 0.0002, 5, 15, 100, 0.25, 0.0002, 20, 6)
8 (False, 0.0002, 7, 5, 60, 0.25, 0.0002, 20, 3)
9 (False, 0.0002, 7, 5, 60, 0.25, 0.0002, 20, 6)
10 (False, 0.0002, 7, 5, 100, 0.25, 0.0002, 20, 3)
11 (False, 0.0002, 7, 5, 100, 0.25, 0.0002, 20, 6)
12 (False, 0.0002, 7, 15, 60, 0.25, 0.0002, 20, 3)
13 (False, 0.0002, 7, 15, 60, 0.25, 0.0002, 20, 6)
14 (False, 0.0002, 7, 15, 100, 0.25, 0.0002, 20, 3)
15 (False, 0.0002, 7, 15, 100, 0.25, 0.0002, 20, 6)
16 (True, 0.0002, 5, 5, 60, 0.25, 0.0002, 20, 3)
17 (True, 0.0002, 5, 5, 60, 0.25, 0.0002, 20, 6)
18 (True, 0.0002, 5, 5, 100, 0.25, 0.0002, 20, 3)
19 (True, 0.0002, 5, 5, 100, 0.25, 0.0002, 20,

In [54]:
gl_p = [False, True] #0
l2_batch = [ 0.0002] #1
n_ker = [5, 7] #2
n_fil = [ 5, 15] #3
d_hidden = [60, 100] #4
drop_out = [0.25] #5
reg_dense = [ 0.0002] #6
min_pool = [ 20] #7
n_lay = [3, 6] #8

# The best hyperparameters for CNN-3, CNN-6, CNN-8

In [35]:
param_best = ((False, 0.0002, 7, 5, 60, 0.25, 0.0002, 20, 3), # CNN-3
         (False, 0.0002, 5, 15, 60, 0.25, 0.0002, 20, 6), # CNN-6
         (False, 0.0001, 3, 10, 100, 0.25, 0.0002, 20, 8)) # CNN-8

In [36]:
inp_shape = X_2_2.shape[1]
inp_shape

124

In [37]:
X_2_2.shape, Y_test_2.shape

((185400, 124), (185400, 2))

In [39]:
x_col = X_2_2.columns
y_col = 'BAD_FLAG'

In [40]:
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )
print(len(x_col), y_col )

124 BAD_FLAG


## Training the best CNN-3, CNN-6, CNN-8

In [42]:
VERBOSE = 2
BATCH_SIZE = 512
NB_EPOCH = 150
class_weighting = {0: 0.05, 1: 1}

result_all_8 = pd.DataFrame()
result_all_8['name_model'] = None
result_all_8['params'] = None
result_all_8['val_GINI'] = None
result_all_8['val_APS'] = None
result_all_8['test_GINI'] = None
result_all_8['test_APS'] = None
j = 0

for p in param_best:
    print('=====================================================================================================')
    print(inp_shape, X_2_2.shape, p)

    model_grid = model_cnn_3(reg=p[1], reg_dense=p[6], n_features=inp_shape, n_pool=2, n_kernel=p[2],
                             n_filters=p[3], n_strides=1, classes=2,
                             hidden=p[4], drop_out=p[5],
                             gl_pool_max=p[0], min_pool=p[7], n_lay=p[8])

    model_grid.compile(loss='categorical_crossentropy',
                       optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
                       metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    history_XX = model_grid.fit_generator(generator=training_generator,  # training_aug,
                                          # Y_test_2 = np_utils.to_categorical( y_val, 2)
                                          validation_data=(X_2_2, Y_test_2),
                                          epochs=NB_EPOCH, verbose=VERBOSE, class_weight=class_weighting,
                                          callbacks=[_time, EarlyStopping(monitor='val_loss', patience=20),
                                                     Metrics(valid_data=(X_2_2, Y_test_2))])

    res_model_ = pd.DataFrame(
        history_XX.history, columns=history_XX.history.keys())
    dd = str(200000 + datetime.now().month*100 +
             datetime.now().day) + '_' + str(datetime.now().hour)
    name_m = 'model_cnn_best_internal_wo_btest_' + str(p[0]) + '_' + str(
        p[2]) + '_' + str(p[3]) + '_' + str(p[4]) + '_' + str(p[8]) + '_' + str(dd)

    model_grid.save(name_m + '.h5')
    res_model_.to_csv(name_m + '.csv')

    predict_class_val = model_grid.predict(X_2_2)
    APS = metrics.average_precision_score(y_test, predict_class_val[:, 1])
    GINI = 2*(metrics.roc_auc_score(y_test, predict_class_val[:, 1])) - 1

    predict_class_test = model_grid.predict(X_3_2)
    APS_t = metrics.average_precision_score( y_val, predict_class_test[:, 1])
    GINI_t = 2*(metrics.roc_auc_score( y_val, predict_class_test[:, 1])) - 1

    result_all_8.at[j, 'name_model'] = name_m
    result_all_8.at[j, 'params'] = str(p)
    result_all_8.at[j, 'val_GINI'] = GINI
    result_all_8.at[j, 'val_APS'] = APS
    result_all_8.at[j, 'test_GINI'] = GINI_t
    result_all_8.at[j, 'test_APS'] = APS_t

    result_all_8.to_csv('model_cnn_internal_wo_btest.csv')
    j += 1
    print(name_m, 'valid_for_train: ', GINI, APS, '| test: ', GINI_t, APS_t)

124 (185400, 124) (False, 0.0002, 7, 5, 60, 0.25, 0.0002, 20, 3)
train_begin | time:  12.05.2021 00:30:51
Epoch 1/150
epoch_end | time:  12.05.2021 00:42:10
 — val_aps:  0.064053 — val_a: 0.907989
2945/2945 - 729s - loss: 0.0182 - accuracy: 0.9524 - auc_2: 0.9868 - precision_2: 0.9524 - recall_2: 0.9524 - val_loss: 0.0311 - val_accuracy: 0.9926 - val_auc_2: 0.9991 - val_precision_2: 0.9926 - val_recall_2: 0.9926
Epoch 2/150
epoch_end | time:  12.05.2021 00:53:35
 — val_aps:  0.062212 — val_a: 0.900581
2945/2945 - 678s - loss: 0.0120 - accuracy: 0.9741 - auc_2: 0.9924 - precision_2: 0.9741 - recall_2: 0.9741 - val_loss: 0.3587 - val_accuracy: 0.8952 - val_auc_2: 0.9195 - val_precision_2: 0.8952 - val_recall_2: 0.8952
Epoch 3/150
epoch_end | time:  12.05.2021 01:04:26
 — val_aps:  0.063386 — val_a: 0.922695
2945/2945 - 646s - loss: 0.0111 - accuracy: 0.9772 - auc_2: 0.9928 - precision_2: 0.9772 - recall_2: 0.9772 - val_loss: 0.0310 - val_accuracy: 0.9938 - val_auc_2: 0.9993 - val_precisi

model_cnn_best_internal_wo_btest_False_7_5_60_3_200512_4 valid_for_train:  0.8653559806867834 0.06508707621943371 | test:  0.8553191646008773 0.055213128910865134
124 (185400, 124) (False, 0.0002, 5, 15, 60, 0.25, 0.0002, 20, 6)
train_begin | time:  12.05.2021 04:44:51
Epoch 1/150
epoch_end | time:  12.05.2021 05:14:22
 — val_aps:  0.067846 — val_a: 0.928973
2945/2945 - 1815s - loss: 0.0179 - accuracy: 0.9738 - auc_3: 0.9940 - precision_3: 0.9738 - recall_3: 0.9738 - val_loss: 0.0195 - val_accuracy: 0.9960 - val_auc_3: 0.9991 - val_precision_3: 0.9960 - val_recall_3: 0.9960
Epoch 2/150
epoch_end | time:  12.05.2021 05:44:04
 — val_aps:  0.077091 — val_a: 0.952237
2945/2945 - 1783s - loss: 0.0073 - accuracy: 0.9855 - auc_3: 0.9974 - precision_3: 0.9855 - recall_3: 0.9855 - val_loss: 0.0260 - val_accuracy: 0.9952 - val_auc_3: 0.9997 - val_precision_3: 0.9952 - val_recall_3: 0.9952
Epoch 3/150
epoch_end | time:  12.05.2021 06:13:28
 — val_aps:  0.056388 — val_a: 0.940225
2945/2945 - 1750s

 — val_aps:  0.060525 — val_a: 0.938509
2945/2945 - 1451s - loss: 0.0046 - accuracy: 0.9889 - auc_3: 0.9991 - precision_3: 0.9889 - recall_3: 0.9889 - val_loss: 0.0278 - val_accuracy: 0.9915 - val_auc_3: 0.9987 - val_precision_3: 0.9915 - val_recall_3: 0.9915
Epoch 27/150
epoch_end | time:  12.05.2021 16:43:16
 — val_aps:  0.058102 — val_a: 0.932877
2945/2945 - 1666s - loss: 0.0046 - accuracy: 0.9893 - auc_3: 0.9991 - precision_3: 0.9893 - recall_3: 0.9893 - val_loss: 0.0562 - val_accuracy: 0.9845 - val_auc_3: 0.9983 - val_precision_3: 0.9845 - val_recall_3: 0.9845
Epoch 28/150
epoch_end | time:  12.05.2021 17:10:11
 — val_aps:  0.054581 — val_a: 0.920552
2945/2945 - 1616s - loss: 0.0045 - accuracy: 0.9892 - auc_3: 0.9991 - precision_3: 0.9892 - recall_3: 0.9892 - val_loss: 0.0362 - val_accuracy: 0.9890 - val_auc_3: 0.9985 - val_precision_3: 0.9890 - val_recall_3: 0.9890
Epoch 29/150
epoch_end | time:  12.05.2021 17:39:47
 — val_aps:  0.058711 — val_a: 0.932275
2945/2945 - 1807s - loss

Epoch 23/150
epoch_end | time:  13.05.2021 01:42:47
 — val_aps:  0.056040 — val_a: 0.937108
2945/2945 - 1194s - loss: 0.0049 - accuracy: 0.9890 - auc_4: 0.9989 - precision_4: 0.9890 - recall_4: 0.9890 - val_loss: 0.0310 - val_accuracy: 0.9947 - val_auc_4: 0.9995 - val_precision_4: 0.9947 - val_recall_4: 0.9947
Epoch 24/150
epoch_end | time:  13.05.2021 02:02:38
 — val_aps:  0.057056 — val_a: 0.938439
2945/2945 - 1190s - loss: 0.0049 - accuracy: 0.9891 - auc_4: 0.9990 - precision_4: 0.9891 - recall_4: 0.9891 - val_loss: 0.0528 - val_accuracy: 0.9866 - val_auc_4: 0.9985 - val_precision_4: 0.9866 - val_recall_4: 0.9866
Epoch 25/150
epoch_end | time:  13.05.2021 02:22:45
 — val_aps:  0.057996 — val_a: 0.942614
2945/2945 - 1206s - loss: 0.0049 - accuracy: 0.9892 - auc_4: 0.9990 - precision_4: 0.9892 - recall_4: 0.9892 - val_loss: 0.0276 - val_accuracy: 0.9921 - val_auc_4: 0.9989 - val_precision_4: 0.9921 - val_recall_4: 0.9921
Epoch 26/150
epoch_end | time:  13.05.2021 02:43:00
 — val_aps: 

## The best CNNs by Profit-metric

In [ ]:
# model_cnn_internal_grid_False_5_10_30_6_200505_13.h5 #cnn-6 best money
# model_cnn_internal_grid_False_5_15_60_8_200508_11.h5 #cnn-8 best money
# model_cnn_internal_grid_False_7_15_60_3_200502_12.h5 #cnn-3 best money

# model_cnn_internal_grid_False_7_5_60_3_200501_13.h5 #cnn-3 best APS

In [70]:
param_best = ((False, 0.0002, 7, 5, 60, 0.25, 0.0002, 20, 3), #cnn-3 best APS
              
         (False, 0.0002, 5, 10, 30, 0.25, 0.0002, 20, 6), #cnn-6 best money
          (False, 0.0002, 5, 15, 60, 0.25, 0.0002, 20, 8) ,  #cnn-8 best money  
              
         (False, 0.0001, 7, 15, 60, 0.25, 0.0002, 20, 3)) #cnn-3 best money

In [71]:
inp_shape = X_2_2.shape[1]
inp_shape

124

In [72]:
X_2_2.shape, Y_test_2.shape

((185400, 124), (185400, 2))

In [73]:
x_col = X_2_2.columns
y_col = 'BAD_FLAG'

In [74]:
training_generator = DataGenerator(train_for, x_col, y_col, batch_size=512, alpha = None, class_w = 0.05 )
print(len(x_col), y_col )

124 BAD_FLAG


### Training CNNs

In [75]:
VERBOSE = 2
BATCH_SIZE = 512
NB_EPOCH = 150
class_weighting = {0: 0.05, 1: 1}

result_all_8 = pd.DataFrame()
result_all_8['name_model'] = None
result_all_8['params'] = None
result_all_8['val_GINI'] = None
result_all_8['val_APS'] = None
result_all_8['test_GINI'] = None
result_all_8['test_APS'] = None
j = 0

for p in param_best:
    print('=====================================================================================================')
    print(inp_shape, X_2_2.shape, p)

    model_grid = model_cnn_3(reg=p[1], reg_dense=p[6], n_features=inp_shape, n_pool=2, n_kernel=p[2],
                             n_filters=p[3], n_strides=1, classes=2,
                             hidden=p[4], drop_out=p[5],
                             gl_pool_max=p[0], min_pool=p[7], n_lay=p[8])

    model_grid.compile(loss='categorical_crossentropy',
                       optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
                       metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    history_XX = model_grid.fit_generator(generator=training_generator,  # training_aug,
                                          # Y_test_2 = np_utils.to_categorical( y_val, 2)
                                          validation_data=(X_2_2, Y_test_2),
                                          epochs=NB_EPOCH, verbose=VERBOSE, class_weight=class_weighting,
                                          callbacks=[_time, EarlyStopping(monitor='val_loss', patience=20),
                                                     Metrics(valid_data=(X_2_2, Y_test_2))])

    res_model_ = pd.DataFrame(
        history_XX.history, columns=history_XX.history.keys())
    dd = str(200000 + datetime.now().month*100 +
             datetime.now().day) + '_' + str(datetime.now().hour)
    name_m = 'model_cnn_best_internal_wo_btest_MONEY_' + str(p[0]) + '_' + str(
        p[2]) + '_' + str(p[3]) + '_' + str(p[4]) + '_' + str(p[8]) + '_' + str(dd)

    model_grid.save(name_m + '.h5')
    res_model_.to_csv(name_m + '.csv')

    predict_class_val = model_grid.predict(X_2_2)
    APS = metrics.average_precision_score(y_test, predict_class_val[:, 1])
    GINI = 2*(metrics.roc_auc_score(y_test, predict_class_val[:, 1])) - 1

    predict_class_test = model_grid.predict(X_3_2)
    APS_t = metrics.average_precision_score( y_val, predict_class_test[:, 1])
    GINI_t = 2*(metrics.roc_auc_score( y_val, predict_class_test[:, 1])) - 1

    result_all_8.at[j, 'name_model'] = name_m
    result_all_8.at[j, 'params'] = str(p)
    result_all_8.at[j, 'val_GINI'] = GINI
    result_all_8.at[j, 'val_APS'] = APS
    result_all_8.at[j, 'test_GINI'] = GINI_t
    result_all_8.at[j, 'test_APS'] = APS_t

    result_all_8.to_csv('model_cnn_internal_wo_btest_MONEY.csv')
    j += 1
    print(name_m, 'valid_for_train: ', GINI, APS, '| test: ', GINI_t, APS_t)

124 (185400, 124) (False, 0.0002, 7, 5, 60, 0.25, 0.0002, 20, 3)
train_begin | time:  14.05.2021 00:16:28
Epoch 1/150
epoch_end | time:  14.05.2021 00:22:57
 — val_aps:  0.057412 — val_a: 0.917690
2945/2945 - 410s - loss: 0.0184 - accuracy: 0.9542 - auc_8: 0.9873 - precision_8: 0.9542 - recall_8: 0.9542 - val_loss: 0.1010 - val_accuracy: 0.9706 - val_auc_8: 0.9913 - val_precision_8: 0.9706 - val_recall_8: 0.9706
Epoch 2/150
epoch_end | time:  14.05.2021 00:29:39
 — val_aps:  0.056694 — val_a: 0.926985
2945/2945 - 401s - loss: 0.0121 - accuracy: 0.9739 - auc_8: 0.9924 - precision_8: 0.9739 - recall_8: 0.9739 - val_loss: 0.2894 - val_accuracy: 0.9128 - val_auc_8: 0.9495 - val_precision_8: 0.9128 - val_recall_8: 0.9128
Epoch 3/150
epoch_end | time:  14.05.2021 00:36:19
 — val_aps:  0.068777 — val_a: 0.931738
2945/2945 - 401s - loss: 0.0108 - accuracy: 0.9770 - auc_8: 0.9936 - precision_8: 0.9770 - recall_8: 0.9770 - val_loss: 0.0482 - val_accuracy: 0.9867 - val_auc_8: 0.9983 - val_precisi

Epoch 27/150
epoch_end | time:  14.05.2021 03:15:01
 — val_aps:  0.067395 — val_a: 0.935312
2945/2945 - 382s - loss: 0.0088 - accuracy: 0.9835 - auc_8: 0.9962 - precision_8: 0.9835 - recall_8: 0.9835 - val_loss: 0.0272 - val_accuracy: 0.9929 - val_auc_8: 0.9994 - val_precision_8: 0.9929 - val_recall_8: 0.9929
Epoch 28/150
epoch_end | time:  14.05.2021 03:21:42
 — val_aps:  0.066938 — val_a: 0.934310
2945/2945 - 400s - loss: 0.0087 - accuracy: 0.9839 - auc_8: 0.9963 - precision_8: 0.9839 - recall_8: 0.9839 - val_loss: 0.0281 - val_accuracy: 0.9940 - val_auc_8: 0.9994 - val_precision_8: 0.9940 - val_recall_8: 0.9940
Epoch 29/150
epoch_end | time:  14.05.2021 03:28:09
 — val_aps:  0.070075 — val_a: 0.937779
2945/2945 - 388s - loss: 0.0086 - accuracy: 0.9841 - auc_8: 0.9963 - precision_8: 0.9841 - recall_8: 0.9841 - val_loss: 0.0897 - val_accuracy: 0.9754 - val_auc_8: 0.9951 - val_precision_8: 0.9754 - val_recall_8: 0.9754
Epoch 30/150
epoch_end | time:  14.05.2021 03:34:49
 — val_aps:  0.

 — val_aps:  0.064423 — val_a: 0.948606
2945/2945 - 1024s - loss: 0.0057 - accuracy: 0.9880 - auc_9: 0.9986 - precision_9: 0.9880 - recall_9: 0.9880 - val_loss: 0.1454 - val_accuracy: 0.9397 - val_auc_9: 0.9876 - val_precision_9: 0.9397 - val_recall_9: 0.9397
Epoch 9/150
epoch_end | time:  14.05.2021 07:40:56
 — val_aps:  0.083948 — val_a: 0.950765
2945/2945 - 1025s - loss: 0.0056 - accuracy: 0.9878 - auc_9: 0.9987 - precision_9: 0.9878 - recall_9: 0.9878 - val_loss: 0.0222 - val_accuracy: 0.9948 - val_auc_9: 0.9995 - val_precision_9: 0.9948 - val_recall_9: 0.9948
Epoch 10/150
epoch_end | time:  14.05.2021 07:57:52
 — val_aps:  0.026378 — val_a: 0.914183
2945/2945 - 1016s - loss: 0.0056 - accuracy: 0.9878 - auc_9: 0.9987 - precision_9: 0.9878 - recall_9: 0.9878 - val_loss: 0.1973 - val_accuracy: 0.9498 - val_auc_9: 0.9810 - val_precision_9: 0.9498 - val_recall_9: 0.9498
Epoch 11/150
epoch_end | time:  14.05.2021 08:15:12
 — val_aps:  0.085860 — val_a: 0.953531
2945/2945 - 1040s - loss:

Epoch 35/150
epoch_end | time:  14.05.2021 15:30:01
 — val_aps:  0.054799 — val_a: 0.930489
2945/2945 - 1484s - loss: 0.0047 - accuracy: 0.9887 - auc_9: 0.9990 - precision_9: 0.9887 - recall_9: 0.9887 - val_loss: 0.0220 - val_accuracy: 0.9952 - val_auc_9: 0.9986 - val_precision_9: 0.9952 - val_recall_9: 0.9952
Epoch 36/150
epoch_end | time:  14.05.2021 15:55:40
 — val_aps:  0.060814 — val_a: 0.917873
2945/2945 - 1546s - loss: 0.0046 - accuracy: 0.9884 - auc_9: 0.9990 - precision_9: 0.9884 - recall_9: 0.9884 - val_loss: 0.0390 - val_accuracy: 0.9897 - val_auc_9: 0.9988 - val_precision_9: 0.9897 - val_recall_9: 0.9897
model_cnn_best_internal_wo_btest_MONEY_False_5_10_30_6_200514_15 valid_for_train:  0.8357454560424373 0.06081387852977906 | test:  0.853276910029289 0.0678130944005578
124 (185400, 124) (False, 0.0002, 5, 15, 60, 0.25, 0.0002, 20, 8)
train_begin | time:  14.05.2021 15:59:57
Epoch 1/150
epoch_end | time:  14.05.2021 16:56:04
 — val_aps:  0.068362 — val_a: 0.937372
2945/2945 

Epoch 24/150
epoch_end | time:  15.05.2021 10:15:50
 — val_aps:  0.051218 — val_a: 0.906138
2945/2945 - 2350s - loss: 0.0045 - accuracy: 0.9897 - auc_10: 0.9991 - precision_10: 0.9897 - recall_10: 0.9897 - val_loss: 0.0225 - val_accuracy: 0.9953 - val_auc_10: 0.9985 - val_precision_10: 0.9953 - val_recall_10: 0.9953
Epoch 25/150
epoch_end | time:  15.05.2021 10:48:09
 — val_aps:  0.066014 — val_a: 0.937481
2945/2945 - 1940s - loss: 0.0045 - accuracy: 0.9895 - auc_10: 0.9991 - precision_10: 0.9895 - recall_10: 0.9895 - val_loss: 0.0234 - val_accuracy: 0.9937 - val_auc_10: 0.9993 - val_precision_10: 0.9937 - val_recall_10: 0.9937
Epoch 26/150
epoch_end | time:  15.05.2021 11:19:32
 — val_aps:  0.043861 — val_a: 0.914250
2945/2945 - 1881s - loss: 0.0045 - accuracy: 0.9898 - auc_10: 0.9991 - precision_10: 0.9898 - recall_10: 0.9898 - val_loss: 0.0454 - val_accuracy: 0.9880 - val_auc_10: 0.9982 - val_precision_10: 0.9880 - val_recall_10: 0.9880
Epoch 27/150
epoch_end | time:  15.05.2021 11:

Epoch 12/150
epoch_end | time:  15.05.2021 19:36:13
 — val_aps:  0.073854 — val_a: 0.952608
2945/2945 - 772s - loss: 0.0066 - accuracy: 0.9863 - auc_11: 0.9982 - precision_11: 0.9863 - recall_11: 0.9863 - val_loss: 0.0200 - val_accuracy: 0.9953 - val_auc_11: 0.9995 - val_precision_11: 0.9953 - val_recall_11: 0.9953
Epoch 13/150
epoch_end | time:  15.05.2021 19:48:58
 — val_aps:  0.068319 — val_a: 0.950767
2945/2945 - 765s - loss: 0.0065 - accuracy: 0.9861 - auc_11: 0.9982 - precision_11: 0.9861 - recall_11: 0.9861 - val_loss: 0.0386 - val_accuracy: 0.9936 - val_auc_11: 0.9992 - val_precision_11: 0.9936 - val_recall_11: 0.9936
Epoch 14/150
epoch_end | time:  15.05.2021 20:01:39
 — val_aps:  0.066342 — val_a: 0.948216
2945/2945 - 760s - loss: 0.0064 - accuracy: 0.9864 - auc_11: 0.9983 - precision_11: 0.9864 - recall_11: 0.9864 - val_loss: 0.0705 - val_accuracy: 0.9861 - val_auc_11: 0.9979 - val_precision_11: 0.9861 - val_recall_11: 0.9861
Epoch 15/150
epoch_end | time:  15.05.2021 20:14: